In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv('final_cleaned.csv')
df.shape


(10070, 2864)

In [3]:
cat_cols = ['company_name','company_about', 'founded', 'business model','employees','product stage','status','fund_stage', 'succeeded']
num_cols = ['total_raised','total_rounds', 'investors','ipo_price', 'geo_market_per']
tag_cols = [col for col in df.columns if col.startswith('tag_')]
targetmarket_cols = [col for col in df.columns if col.startswith('targetmarket_')]
sector_list = [col for col in df.columns if col.startswith("sector_")]
target_ind_list = [col for col in df.columns if col.startswith("target_industry_")]
technology_list = [col for col in df.columns if col.startswith("core_technology_")]
bin_cols = tag_cols + targetmarket_cols + sector_list + target_ind_list + technology_list

In [4]:

new_tech_cols = []
new_industry_cols = []
for col in target_ind_list + technology_list:
    if col in technology_list:
        new_tech_cols.append("technology" + col[15:])
    
    elif col in target_ind_list:
        new_industry_cols.append("industry" + col[15:])


d= {}
for i in range(len(new_tech_cols)):
    d[technology_list[i]] =  new_tech_cols[i]

for i in range(len(new_industry_cols)):
    d[target_ind_list[i]] =  new_industry_cols[i]

df.rename(columns=d, inplace=True)


In [5]:
target_ind_list = [col for col in df.columns if col.startswith("industry_")]
technology_list = [col for col in df.columns if col.startswith("technology_")]
bin_cols = tag_cols + targetmarket_cols + sector_list + target_ind_list + technology_list

In [6]:
m = {}
for l in [tag_cols, targetmarket_cols, sector_list, target_ind_list, technology_list]:
    subject = l[0].split("_")[0]
    temp = []
    for col in l:
        temp.append(df[col].sum())
    m[subject] = sorted(temp, reverse=True)


In [20]:
outliers_dict = {}
for subject, l in m.items():

    Q1 = np.percentile(l, 25)
    Q3 = np.percentile(l, 75)
    IQR = 1.5*(Q3 - Q1)

    lower_bound = Q1 - IQR
    upper_bound = Q3 + IQR

    outlier_list = [x for x in l if x < lower_bound or x > upper_bound]
    outliers_dict[subject] = outlier_list

print("Outliers:")
for key, val in outliers_dict.items():
    print(key, len(val))

# df.loc[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR), col] = np.nan


Outliers:
tag 283
targetmarket 33
sector 4
industry 9
technology 15


In [44]:
print(outliers_dict['tag'])

[2173.0, 2029.0, 1391.0, 1387.0, 1273.0, 1152.0, 1091.0, 1007.0, 868.0, 774.0, 769.0, 768.0, 743.0, 687.0, 684.0, 649.0, 640.0, 625.0, 612.0, 608.0, 605.0, 597.0, 558.0, 557.0, 529.0, 526.0, 510.0, 467.0, 431.0, 425.0, 407.0, 399.0, 387.0, 386.0, 384.0, 377.0, 368.0, 351.0, 351.0, 349.0, 344.0, 342.0, 340.0, 336.0, 333.0, 332.0, 326.0, 317.0, 316.0, 315.0, 315.0, 313.0, 313.0, 309.0, 303.0, 300.0, 294.0, 292.0, 291.0, 290.0, 288.0, 282.0, 277.0, 276.0, 270.0, 261.0, 261.0, 258.0, 257.0, 252.0, 246.0, 242.0, 241.0, 239.0, 235.0, 234.0, 231.0, 231.0, 230.0, 229.0, 228.0, 228.0, 225.0, 223.0, 213.0, 210.0, 209.0, 206.0, 203.0, 202.0, 198.0, 198.0, 198.0, 197.0, 196.0, 190.0, 188.0, 187.0, 186.0, 176.0, 174.0, 173.0, 172.0, 171.0, 169.0, 169.0, 169.0, 169.0, 167.0, 167.0, 166.0, 166.0, 165.0, 165.0, 163.0, 162.0, 161.0, 160.0, 158.0, 157.0, 156.0, 153.0, 153.0, 152.0, 151.0, 151.0, 151.0, 151.0, 150.0, 149.0, 149.0, 149.0, 149.0, 147.0, 146.0, 145.0, 144.0, 144.0, 143.0, 143.0, 143.0, 143.

In [10]:
df_bin = pd.DataFrame(columns = ['subject','binary_col', 'count'])
for l in [tag_cols, targetmarket_cols, sector_list, target_ind_list, technology_list]:
    for col in l:
        df_bin = df_bin.append({'subject': col.split('_')[0],'binary_col': col , 'count': df[col].sum()}, ignore_index=True)

df_bin

,subject,binary_col,count
0,tag,tag_computer-vision,309.0
1,tag,tag_behavior-analytics,351.0
2,tag,tag_machine-learning,1152.0
3,tag,tag_consumer-packaged-goods,8.0
4,tag,tag_natural-language-processing,294.0
...,...,...,...
2845,technology,technology_Materials & Substances_Nonwoven Fabric,5.0
2846,technology,technology_Materials & Substances_Insulation,5.0
2847,technology,technology_Sensing_Scent,4.0
2848,technology,technology_Machinery & Robotics_Exoskeleton,1.0


In [26]:

df_cor = df[tag_cols].corr()

In [41]:
df_cor.shape

(2340, 2340)

In [85]:
a = df[df['tag_astrology']== 1]
for row in a.iterrows():
    for i, r in enumerate(row[1]):
        if r == 1:
            print(a.columns[i], r)
#     for col in a.columns:
        # if row.loc[0,col] == 1:
        # if a.loc[0, col] == 1:
    # print(type(row[1]))

product stage 1
tag_hobbies 1.0
tag_numerology 1.0
tag_horoscope 1.0
tag_graphology 1.0
tag_psychic 1.0
tag_medium 1.0
tag_spirit 1.0
tag_tarot 1.0
tag_astrology 1.0
tag_alternative-energy 1.0
tag_clairvoyance 1.0
tag_fortune-teller 1.0
sector_Content & Media 1.0
sector_Content & Media_Media & Entertainment Tech 1.0
industry_Media & Entertainment 1.0
industry_Media & Entertainment_Social Media 1.0
technology_Platforms & Interfaces 1.0
technology_Platforms & Interfaces_Web 1.0


In [43]:
# find highest values in dataframe
e = []
for i in range(len(df_cor.columns)):
    for j in range(i+1, len(df_cor.columns)):
        if abs(df_cor.iloc[i,j]) > 0.7:
            e.append((df_cor.columns[i], df_cor.columns[j]))
            # print(df_cor.columns[i], df_cor.columns[j], df_cor.iloc[i,j])   


#  create a graph with v and e
# e = [(df_cor.columns[i], df_cor.columns[j]) for i in range(len(df_cor.columns)) for j in range(len(df_cor.columns)) if df_cor.iloc[i,j] != 1 and abs(df_cor.iloc[i,j]) > 0.7]



tag_climate-tech tag_sdg 0.7155656694533937
tag_urban-recue tag_work-at-height 0.7070716672557408
tag_urban-recue tag_rope-access 0.7070716672557408
tag_urban-recue tag_high-angled-rope-rescue 0.7070716672557408
tag_astronautics tag_astrospace 0.864933448700209
tag_electro-chemical tag_impact-investing 0.707071667255741
tag_plant-climate-management tag_climate-change 0.7070716672557411
tag_flood-irrigation tag_farming-equipment 1.0
tag_fungicides tag_herbicides 0.8486274125880997
tag_spirulina tag_shrimp-farms 0.7070716672557399
tag_spirulina tag_enery-to-food- 1.0
tag_spirulina tag_omega-3 1.0
tag_shrimp-farms tag_enery-to-food- 0.7070716672557399
tag_shrimp-farms tag_omega-3 0.7070716672557399
tag_enery-to-food- tag_omega-3 1.0
tag_cloning tag_planting 0.7070716672557408
tag_thinning tag_tractors 0.7070716672557398
tag_mastitis tag_protein-based 0.7070716672557402
tag_waterworks tag_hydraulic-valves 0.7070365446051793
tag_satellite-modem tag_data-communication 0.7070716672557408
tag_

In [58]:
index = [0]*len(tag_cols)

for edge in e:
    index[tag_cols.index(edge[0])] +=1
    index[tag_cols.index(edge[1])] +=1

l = [(tag_cols[i], index[i]) for i in range(len(tag_cols)) if index[i] > 0]
l.sort()

# max(index)

In [87]:
max = 0
for t in l:
    if t[1] > max:
        max = t[1]

max

9

In [49]:
subjects = ['tag','targetmarket','sector','industry','technology']
print("Z score:")
scores = []
for subject in subjects:

    z_score = (df_bin[df_bin.subject == subject]['count'] - df_bin[df_bin.subject == subject]['count'].mean()) / df_bin[df_bin.subject == subject]['count'].std()
    outliers = abs(z_score) > 3
    scores.append(outliers)
    print(f'{subject}: {sum(outliers)}')

df_bin[df_bin.subject == 'tag'][scores[0]]

Z score:
tag: 32
targetmarket: 5
sector: 1
industry: 3
technology: 2


,subject,binary_col,count
2,tag,tag_machine-learning,1152.0
6,tag,tag_personalization,597.0
7,tag,tag_artificial-intelligence,2029.0
8,tag,tag_data-analytics,605.0
21,tag,tag_climate-tech,768.0
23,tag,tag_sdg,1391.0
28,tag,tag_agtech,557.0
32,tag,tag_sensors,612.0
34,tag,tag_iot,608.0
35,tag,tag_big-data,431.0


In [ ]:
df[].corr()

In [ ]:
outliers_dict['tag']

In [ ]:
# sum of each column in dataframe
list_target_industry_dict = {}
list_target_industry = []
count = 0
for col in sector_list:
    list_target_industry_dict[col] = df[col].sum()
    list_target_industry.append(list_target_industry_dict[col])
    if list_target_industry_dict[col] == 1 :
        count += 1
    # print(f'{col}: {df[col].sum()}')
print(count)
# sort dict values in descending order
list_target_industry2 = sorted(list_target_industry_dict.items(), key=lambda x: x[1])
list_target_industry2

In [ ]:
print(f"avg: {sum(list_target_industry_dict.values()) / len(list_target_industry_dict)}")
print(f"median: {sorted(list_target_industry_dict.values())[int(len(list_target_industry_dict)/4)]}")

In [ ]:
#  plot the values of list

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))


plt.bar(list_target_industry_dict.keys(), list_target_industry_dict.values())